# Tokenizers

Compariamo alcuni LLM tokenizers pre-addestrati

In [2]:
!pip install transformers>=4.46.1

## Tokenizing Text

Tokenizzeremo la frase "Hello world!" usando il tokenizer del modello **bert-base-cased**.

Importiamo la classe **Autotokenizer**.

In [5]:
from transformers import AutoTokenizer

In [7]:
sentence = "Hello world!"

In [9]:
# carichiamo il tokenizer pre-addestrato
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Applichiamo il tokenizer alla frase. Il tokenizer splitta la frase in tokens and returns the IDs of each token.

In [12]:
token_ids = tokenizer(sentence).input_ids

In [14]:
print(token_ids)

[101, 8667, 1362, 106, 102]


Per mappare ciascun token ID al suo corrispondente token, possiamo usare il metodo **decode** del tokenizer

In [17]:
for id in token_ids:
    print(tokenizer.decode(id))

[CLS]
Hello
world
!
[SEP]


## Visualizziamo la tokenizzazione

Involucriamo gli step precedenti dentro una funzione **show_tokens**.
La funzione prende in input un testo e il nome del modello, e stampa la lunghezza del vocabolario del tokenizer e una lista colorata dei tokens.

In [20]:
# Una lista di colori in RGB per evidenziare i tokens e ci aiuta a differenziare tokens diversi
colors = [
    "102;194;165", "252;141;98", "141;160;203",
    "231;138;195", "166;216;84", "255;217;47"
]

def show_tokens(sentence: str, tokenizer_name: str):
    """Show the tokens each separated by a different color."""

    # carichiamo il tokenizer e tokenizziamo l'input
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids

    # stampiamo la lunghezza del vocabolario
    print(f"Vocab length: {len(tokenizer)}")

    # stampa una lista colorata di tokens
    for idx, t in enumerate(token_ids):
        print(
            f"\x1b[0;30;48;2;{colors[idx % len(colors)]}m" +
            tokenizer.decode(t) + 
            "\x1b[0m",
            end=" "
        )

Qui vedimao il testo che useremo per esplorare le diverse strategie di tokenizzazione di ciascun modello

In [23]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "      "
12.0*50=600
"""

Ora usiamo di nuovo il tokenizer del modello bert-base-cased e compariamo la sua strategia di tokenizzazione a quella di **Xenova/gpt-4** 

In [26]:
# bert-base-cased
show_tokens(text, "bert-base-cased")

Vocab length: 28996
[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

Vediamo che il tokenizer del modello bert-base-cased ha difficoltà nel rappresentare la parola CAPITALIZATION. Infatti notiamo come viene splittata in più tokens. Gli hashtag mostrano che tali tokens appartengono al token precedente e che assieme essi rappresentano una singola parola. Il token **UNK** rapprsenta il token Unknown

In [29]:
# Usiamo il tokenizer del modello GPT-4, chiamato Tiktoken
show_tokens(text, "Xenova/gpt-4")

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Vocab length: 100263

 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "        "
 12 . 0 * 50 = 600 
 

Vediamo che il tokenizer Tiktoken del modello gpt-4 è molto più grande del tokenizer del modello bert-base-cased con una lunghezza di 100263 tokens. Tiktoken necessita di meno tokens per rappresentare gli inputs. Vediamo come la parola CAPITALIZATION ora è stata splittata in soli due tokens. È interessante vedere come necessita meno tokens per rappresentare gli inputs, anche i tabs gli rapprsenta molto meglio. DAto che ha un vocabolario molto più grande è più semplice rappresentare tokens meno comuni, ma però c'è un trade-off. Più grande è il vocabolario, più embeddings devono essere calcolati per ciascuno di questi tokens. Quindi c'è un trade-off tra scegliere un tokenizer con potenzialmente una lunghezza di milioni di tokens, e imparare la rappresentazione (embeddings) per ciascuno di questi tokens.